# Predicción de NBV 

En este notebook implementaremos un sistema para reconstrucción via NBV haciendo uso de la arquitectura autoencoder y MLP

* pedirá la dirección a la carpeta contenedora del objeto
* Creará un link simbolico para acceder a las texturas (verifica si no existe ya) y carpetas para almacenar la información obtenida durante el proceso: 
    - Nubes de puntos
    - RGB
    - Profundidad
* Despliega el objeto para verificar si es el que espera el usuario y debe confirmar si es correcto
* Genera una posición y orientación random (podría extraerla de Hintertoiser), importante conocer el bounding box para no colisionar con el objeto
* Inicia el proceso de reconstrucción
* Captura información y almacena 
* Crea grid de 31x31x31
* Procesamiento de IA
* Condición si la cumple repite o finaliza
* Reporte de métricas

## TODO: guarda y visualizar nube de puntos e imagen

In [1]:
import os
import open3d as o3d
import octomap
import torch
import numpy as np
from symlink import symbolic_dir
from utils_o3d import Get_Pointcloud, Get_RGBD, Get_octree, Get_PointcloudGT, scale_and_translate, Get_voxpoints
from MLP import MLP
from utils import net_position_nbv
from dataset_download import download_collection
from utils_metrics import chamfer_distance, Get_surface_coverage, stop_condition

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#pages is the number of objects you want to download
download_collection(owner_name="GoogleResearch", collection_name="Scanned Objects by Google Research", folder="obj", pages=2)

Folder already exist, verify if dataset is downloaded


In [2]:
if os.path.lexists("poses.npy") == False:
    from tesselation import hinter_sampling
    poses, _ = hinter_sampling(50, radius = 1)
    np.save("poses.npy", poses)
    print("teselando")
#print("Ingresa la dirección a la carpeta contenedora del banco de datos")
#direccion = input("Cúal es la dirección a la carpeta contenedora?:") # Dataset acces
direccion = 'objetos'
x = os.listdir(direccion)
carpeta = input("A que carpeta quieres acceder?: ") #object folder
direccion_disco = '/mnt/6C24E28478939C77/Saulo/ProyectoPHD/'
dir_carpeta = direccion_disco + direccion #+ "/" + carpeta
if os.path.lexists(dir_carpeta + "/meshes/texture.png") == False:
    symbolic_dir(dir_carpeta)
    RGB = "/RGB"
    Depth = "/Depth"
    Point_cloud = "/Point_cloud"
    Octree = "/Octree"
    os.mkdir(dir_carpeta + RGB)
    os.mkdir(dir_carpeta + Depth)
    os.mkdir(dir_carpeta + Point_cloud)
    os.mkdir(dir_carpeta + Octree)
visualizar = input("Quieres visualizar el objeto? (presiona S en caso de sí):") 
if visualizar == 'S'  or  visualizar == 's' :
    mesh = o3d.io.read_triangle_mesh(dir_carpeta + '/meshes/model.obj', True) 
    orig = o3d.geometry.TriangleMesh.create_coordinate_frame()
    o3d.visualization.draw_geometries([mesh])

In [ ]:
if os.path.lexists("poses.npy") == False:
    from tesselation import hinter_sampling
    poses, _ = hinter_sampling(50, radius = 1)
    np.save("poses.npy", poses)
    print("teselando")
#print("Ingresa la dirección a la carpeta contenedora del banco de datos")
#direccion = input("Cúal es la dirección a la carpeta contenedora?:") # Dataset acces
direccion = 'objetos'
x = os.listdir(direccion)
carpeta = input("A que carpeta quieres acceder?: ") #object folder
direccion_disco = '/mnt/6C24E28478939C77/Saulo/ProyectoPHD/'
dir_carpeta = direccion_disco + direccion #+ "/" + carpeta
if os.path.lexists(dir_carpeta + "/meshes/texture.png") == False:
    symbolic_dir(dir_carpeta)
    RGB = "/RGB"
    Depth = "/Depth"
    Point_cloud = "/Point_cloud"
    Octree = "/Octree"
    os.mkdir(dir_carpeta + RGB)
    os.mkdir(dir_carpeta + Depth)
    os.mkdir(dir_carpeta + Point_cloud)
    os.mkdir(dir_carpeta + Octree)
visualizar = input("Quieres visualizar el objeto? (presiona S en caso de sí):") 
if visualizar == 'S'  or  visualizar == 's' :
    mesh = o3d.io.read_triangle_mesh(dir_carpeta + '/meshes/model.obj', True) 
    orig = o3d.geometry.TriangleMesh.create_coordinate_frame()
    o3d.visualization.draw_geometries([mesh])

img_W = 100
img_H = 100
#Cargamos los modelos de predicción de posición
model= MLP().cuda() 
path_weights = '/mnt/6C24E28478939C77/Saulo/ProyectoPHD/position/weights_entrenamiento_MLP_xavier_normal_2.pth' ## Modificar direccion de pesos
model.load_state_dict(torch.load(path_weights))
device = torch.cuda.current_device()

#Inicializamos el octomap
resolution = 0.01 # resolucion del octree
octree = octomap.OcTree(resolution) # inicializamos el octree

#Cargamos malla
mesh = o3d.io.read_triangle_mesh(dir_carpeta + '/meshes/model.obj', True)
material = o3d.visualization.rendering.MaterialRecord() # Create material
material.albedo_img = o3d.io.read_image( dir_carpeta + '/meshes/texture.png') # Add texture
mesh = scale_and_translate(mesh, scale_factor=0.39)
#Obtenemos pointcloud GT
Get_PointcloudGT(dir_carpeta, mesh)

# Raycasting
mesh1 = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(mesh1)
# render for RGBD images
render = o3d.visualization.rendering.OffscreenRenderer(width=img_W, height=img_H) #Linux only
render.scene.add_geometry('mesh', mesh, material)
#Camera vectors setup
cent = [0,0,.2]#mesh.get_center()
up = [0, 1, 0]
poses = np.load("poses.npy")
eye = [-1,-1,.5]
fov = 25
puntos = Get_voxpoints()
#occupancy_probs = np.ndarray([29791], dtype=float)
#occupancy_probs = np.full((29791), 0.5)

metricas = {"objeto": [], "chamfer": [], "distancia": [], "vistas": []}
metricas["objeto"].append(carpeta)
print("Inicia el proceso de reconstrucción ...")
#while condicion == False:
for i in range(0,15):    

    # RGBD and pointcloud extraction
    Get_Pointcloud(scene, fov, cent, eye, up, img_W, img_H, dir_carpeta, i)
    Get_RGBD(render,  fov, cent, eye, up, dir_carpeta, i)
    #Occupancy grid
    occupancy_probs =  Get_octree(octree, dir_carpeta, i, eye, puntos) #occupancy_probs)
    direccion_octree= bytes(dir_carpeta + "/Octree/octree_{}.ot".format(i), encoding='utf8')
    octree.writeBinary(direccion_octree) 
    np.save(dir_carpeta + "/Octree/grid_{}.npy".format(i),occupancy_probs, allow_pickle=True)
    ## Aqui evaluamos si esta completo el modelo en este punto
    CD = chamfer_distance(dir_carpeta)
    if i > 0:
        ind, _ = Get_surface_coverage(dir_carpeta, i)
        condicion, Distance = stop_condition(dir_carpeta,ind,i)
    else: 
        condicion = False
        Distance = 0.0
    print("Chamfer Distance: {}, Cloud distances: {}, # view: {}".format(CD, Distance, i))
    metricas["chamfer"].append(CD)
    metricas["distancia"].append(Distance)
    if condicion == True:
        break
    ## De no estarlo, se consulta a la NN el NBV 
    else:
        grid = np.reshape(occupancy_probs, (1,1,31,31,31))  
        torch_grid = torch.from_numpy(grid)
        #IA-NBV
        output = net_position_nbv(model, torch_grid, device) 
        eye = output.numpy().reshape(3,).astype("double") 
    print("nbv:", eye)

metricas["vistas"].append(i)
print(metricas)   
print("Volví, tonotos!")
#almacena las métricas de error en archivo NPZ
np.savez('metricas.npz', obj=metricas["objeto"] ,chamfer = metricas["chamfer"], distancias = metricas["distancia"], numero=metricas["vistas"])

In [ ]:
o3d.visualization.draw_geometries([mesh])

## Pruebas de MAtrices

In [ ]:
import open3d as o3d
import numpy as np 
from utils_camera import save_camera_trayectory

In [ ]:
eye = np.load("poses.npy")
center = [0,0,.2]#mesh.get_center()
up = [0, 1, 0]
fov = 25
width = 100
height = 100
direccion = "objetos"

In [ ]:
save_camera_trayectory(direccion, eye[:20], center, up, fov, width, height)

## Pruebas ploteo trayectoria

In [1]:
import open3d as o3d
import numpy as np
#from utils_o3d import camara
from utils_visualizer import draw_camera
#param = o3d.io.read_pinhole_camera_parame]ters("viewpoint.json")
#ctr.convert_from_pinhole_camera_parameters(param)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
direccion = "objetos"
param = o3d.io.read_pinhole_camera_trajectory(direccion + "/trayectoria_camara.json")

In [3]:
frames = []
scale = 1.0
color=[0.8,0.2,0.8]
for i in range (0,len(param.parameters)):
    extrinsic = param.parameters[i].extrinsic
    intrinsic = param.parameters[i].intrinsic.intrinsic_matrix
    width = param.parameters[i].intrinsic.width
    height = param.parameters[i].intrinsic.height
    camera_model = draw_camera(intrinsic, extrinsic, width, height)
    frames.extend(camera_model)
    #for i in frames:
    #    self.__vis.add_geometry(i)


In [4]:
mesh = o3d.io.read_triangle_mesh('obj/Marvel_Avengers_Titan_Hero_Series_Doctor_Doom/meshes/model.obj', True) 
frames.append(mesh)
o3d.visualization.draw_geometries(frames)

## Extras

In [ ]:
import open3d as o3d
import numpy as np
import octomap
import trimesh
import matplotlib.pyplot as plt
import cv2
from utils_o3d import scale_and_translate, Get_voxpoints

In [ ]:
from classification_nbv import showGrid, showGrid4
import numpy as np

In [ ]:
occupancy_probs = np.load('/mnt/6C24E28478939C77/Saulo/ProyectoPHD/objetos'+'/Octree/grid_6.npy')

In [ ]:
model= MLP().cuda() 
path_weights = '/mnt/6C24E28478939C77/Saulo/ProyectoPHD/position/weights_entrenamiento_MLP_xavier_normal_2.pth' ## Modificar direccion de pesos
model.load_state_dict(torch.load(path_weights))
device = torch.cuda.current_device()

In [ ]:
grid = np.reshape(occupancy_probs, (1,1,31,31,31))  
torch_grid = torch.from_numpy(grid)
#IA-NBV
output = net_position_nbv(model, torch_grid, device)
output

In [ ]:
occupancy_probs1 = np.ndarray([29791], dtype=float)
print(occupancy_probs1)

In [ ]:
np.set_printoptions(threshold = np.inf,precision=4, suppress=True)
print(occupancy_probs)

In [ ]:
showGrid(oc_prob, "1.png")

In [ ]:
showGrid4(occupancy_probs)

In [ ]:
mesh.get_axis_aligned_bounding_box()

In [ ]:
del octree
del mesh
del mesh1
del material
del scene
del render

In [ ]:
del scene
del render

## Reconstrucción pato (dataset entrenamiento)

In [ ]:
mesh = o3d.io.read_triangle_mesh('duck/mesh.ply')
o3d.visualization.draw_geometries([mesh])

In [ ]:
if os.path.lexists("poses.npy") == False:
    from tesselation import hinter_sampling
    poses, _ = hinter_sampling(50, radius = 1)
    np.save("poses.npy", poses)
    print("teselando")
#print("Ingresa la dirección a la carpeta contenedora del banco de datos")
#direccion = input("Cúal es la dirección a la carpeta contenedora?:") # Dataset acces
direccion = 'duck'
x = os.listdir(direccion)
#carpeta = input("A que carpeta quieres acceder?: ") #object folder
direccion_disco = '/mnt/6C24E28478939C77/Saulo/ProyectoPHD/'
dir_carpeta = direccion_disco + direccion #+ "/" + carpeta
#symbolic_dir(dir_carpeta)
#RGB = "/RGB"
#Depth = "/Depth"
#Point_cloud = "/Point_cloud"
#Octree = "/Octree"
#os.mkdir(dir_carpeta + RGB)
#os.mkdir(dir_carpeta + Depth)
#os.mkdir(dir_carpeta + Point_cloud)
#os.mkdir(dir_carpeta + Octree)
visualizar = input("Quieres visualizar el objeto? (presiona S en caso de sí):") 
if visualizar == 'S'  or  visualizar == 's' :
    mesh = o3d.io.read_triangle_mesh(dir_carpeta + '/mesh.ply', True) 
    o3d.visualization.draw_geometries([mesh])

img_W = 100
img_H = 100
#Cargamos los modelos de predicción de posición
model= MLP().cuda() 
path_weights = '/mnt/6C24E28478939C77/Saulo/ProyectoPHD/position/weights_entrenamiento_MLP_xavier_normal_2.pth' ## Modificar direccion de pesos
model.load_state_dict(torch.load(path_weights))
device = torch.cuda.current_device()

#Inicializamos el octomap
resolution = 0.01 # resolucion del octree
octree = octomap.OcTree(resolution) # inicializamos el octree

#Cargamos malla
mesh = o3d.io.read_triangle_mesh(dir_carpeta + '/mesh.ply', True)
material = o3d.visualization.rendering.MaterialRecord() # Create material
material.albedo_img = o3d.io.read_image( dir_carpeta + '/texture.png') # Add texture
max = mesh.get_max_bound()
scale = 0.4/np.max(max)
mesh.scale(scale, center= [0,0,.2])#Scale mesh
traslacion = [-.1,-.1,0]- np.asarray(mesh.get_min_bound())
#traslacion =  [0,0,traslacion]
mesh.translate(traslacion) #translate mesh
max = mesh.get_max_bound()
scale = 0.4/np.max(max)
mesh.scale(scale, center= [0,0,.2])#Scale mesh
traslacion = [-.18,-.18,0]- np.asarray(mesh.get_min_bound())
#traslacion =  [0,0,traslacion]
mesh.translate(traslacion) #translate mesh
#Obtenemos pointcloud GT
Get_PointcloudGT(dir_carpeta, mesh)

# Raycasting
mesh1 = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(mesh1)
# render for RGBD images
render = o3d.visualization.rendering.OffscreenRenderer(width=img_W, height=img_H) #Linux only
render.scene.add_geometry('mesh', mesh, material)
#Camera vectors setup
cent = mesh.get_center()
up = [0, 1, 0]
poses = np.load("poses.npy")
eye = [-1,-1,.5]
fov = 30
puntos = Get_voxpoints()
#occupancy_probs = np.ndarray([29791], dtype=float)
#occupancy_probs = np.full((29791), 0.5)

metricas = {"objeto": [], "chamfer": [], "distancia": [], "vistas": []}
metricas["objeto"].append(direccion)
print("Inicia el proceso de reconstrucción ...")
#while condicion == False:
for i in range(0,20):    

    # RGBD and pointcloud extraction
    Get_Pointcloud(scene, fov, cent, eye, up, img_W, img_H, dir_carpeta, i)
    Get_RGBD(render,  fov, cent, eye, up, dir_carpeta, i)
    #Occupancy grid
    occupancy_probs =  Get_octree(octree, dir_carpeta, i, eye, puntos) #occupancy_probs)
    direccion_octree= bytes(dir_carpeta + "/Octree/octree_{}.ot".format(i), encoding='utf8')
    octree.writeBinary(direccion_octree) 
    np.save(dir_carpeta + "/Octree/grid_{}.npy".format(i),occupancy_probs, allow_pickle=True)
    ## Aqui evaluamos si esta completo el modelo en este punto
    CD = chamfer_distance(dir_carpeta)
    if i > 0:
        ind, _ = Get_surface_coverage(dir_carpeta, i)
        condicion, Distance = stop_condition(dir_carpeta,ind,i)
    else: 
        condicion = False
        Distance = 0.0
    print("Chamfer Distance: {}, Cloud distances: {}, # view: {}".format(CD, Distance, i))
    metricas["chamfer"].append(CD)
    metricas["distancia"].append(Distance)
    if condicion == True:
        break
    ## De no estarlo, se consulta a la NN el NBV 
    else:
        grid = np.reshape(occupancy_probs, (1,1,31,31,31))  
        torch_grid = torch.from_numpy(grid)
        #IA-NBV
        output = net_position_nbv(model, torch_grid, device) 
        eye = output.numpy().reshape(3,).astype("double") 
    print("nbv:", eye)

metricas["vistas"].append(i)
print(metricas)   
print("Volví, tonotos!")
#almacena las métricas de error en archivo NPZ
np.savez('metricas_duck.npz', obj=metricas["objeto"] ,chamfer = metricas["chamfer"], distancias = metricas["distancia"], numero=metricas["vistas"])

In [ ]:
mesh.get_axis_aligned_bounding_box()

In [ ]:
import open3d as o3d

In [ ]:
nube = o3d.io.read_point_cloud('/mnt/6C24E28478939C77/Saulo/ProyectoPHD/obj/'+'2_of_Jenga_Classic_Game'+"/Point_cloud/cloud_0.pcd")

In [ ]:
del mesh

In [ ]:
mesh = o3d.io.read_triangle_mesh('duck/mesh.ply')

In [ ]:
mesh.get_axis_aligned_bounding_box()

In [ ]:
max = mesh.get_max_bound()
scale = 0.4/np.max(max)
mesh.scale(scale, center= [0,0,.2])#Scale mesh
traslacion = [-.1,-.1,0]- np.asarray(mesh.get_min_bound())
#traslacion =  [0,0,traslacion]
mesh.translate(traslacion) #translate mesh
max = mesh.get_max_bound()
scale = 0.4/np.max(max)
mesh.scale(scale, center= [0,0,.2])#Scale mesh
traslacion = [-.1,-.1,0]- np.asarray(mesh.get_min_bound())
#traslacion =  [0,0,traslacion]
mesh.translate(traslacion) #translate mesh
mesh.get_axis_aligned_bounding_box()

In [ ]:






mesh.get_max_bound()

In [ ]:
o3d.visualization.draw_geometries([mesh])

In [ ]:
traslacion = - np.asarray(mesh.get_min_bound())[2]
traslacion =  [0,0,traslacion]
mesh.translate(traslacion) #translate mesh
o3d.visualization.draw_geometries([mesh])

In [ ]:
mesh.get_axis_aligned_bounding_box()

In [ ]:
mesh.get_center()

In [ ]:
import open3d as o3d
import numpy as np

In [ ]:
demo_crop_data = o3d.data.DemoCropPointCloud()
pcd = o3d.io.read_point_cloud(demo_crop_data.point_cloud_path)
vol = o3d.visualization.read_selection_polygon_volume(demo_crop_data.cropped_json_path)
chair = vol.crop_point_cloud(pcd)

dists = pcd.compute_point_cloud_distance(chair)
dists = np.asarray(dists)
ind = np.where(dists > 0.01)[0]
pcd_without_chair = pcd.select_by_index(ind)
o3d.visualization.draw_geometries([pcd_without_chair],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [ ]:
dists1 = pcd.compute_mahalanobis_distance()
dists1 = np.asarray(dists1)
ind1 = np.where(dists1 > 0.01)[0]
pcd_without_chair1 = pcd.select_by_index(ind1)
o3d.visualization.draw_geometries([pcd_without_chair1],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [ ]:
dists1 = pcd.compute_nearest_neighbor_distance()
dists1 = np.asarray(dists1)
ind1 = np.where(dists1 > 0.004)[0]
pcd_without_chair1 = pcd.select_by_index(ind1)
o3d.visualization.draw_geometries([pcd_without_chair1],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [ ]:
p_c = o3d.io.read_point_cloud("obj/2_of_Jenga_Classic_Game/Point_cloud/cloud_acc.pcd")
#o3d.visualization.draw_geometries([p_c])

In [ ]:
pcd = o3d.io.read_point_cloud("obj/2_of_Jenga_Classic_Game/Point_cloud/cloud_8.pcd")

In [ ]:
p_c.Type

In [ ]:
size = np.asarray(mesh.vertices).shape[0]

In [ ]:
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])
pcd = mesh.sample_points_uniformly(number_of_points=size*10)
o3d.visualization.draw_geometries([pcd])

In [ ]:
pcd.Type

In [ ]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
np.asarray(pcd.points).shape

In [ ]:
dists.shape

In [ ]:
len(p_c.points)

In [ ]:
len(pcd.points)

In [ ]:
dist = np.asarray(p_c.compute_point_cloud_distance(p_c))
ind = np.where(dist < 0.0037)[0]
pcd_wo = p_c.select_by_index(ind, invert=True) 
o3d.visualization.draw_geometries([pcd_wo])

In [ ]:
len(pcd_wo.points)

In [ ]:
np.savez('/kaggle/working/entrenamiento_mlp_7.npz', entrenamiento = losses["entrenamiento"], validación = losses["validación"])

In [ ]:
direccion_disco = '/mnt/6C24E28478939C77/Saulo/ProyectoPHD/obj/2_of_Jenga_Classic_Game'

In [ ]:
Get_cloud_distance(direccion_disco,9, umbral= 0.0037)

In [ ]:
chamfer_distance(np.asarray(p_c.points),np.asarray(pcd.points))

In [ ]:
losses = {"objeto": [], "chamfer": [], "distancia": [], "vistas": []}

In [ ]:
arrays = np.zeros((15))
arrys1 = np.ones((15))
vista = 5


In [ ]:
losses["objeto"].append("primero")
losses["chamfer"].append(arrays[vista*0:vista*1])
losses["distancia"].append(arrys1[vista*0:vista*1])
losses["vistas"].append(vista)

In [ ]:
losses["objeto"].append("segundo")
losses["chamfer"].append(arrays[vista*1:vista*2])
losses["distancia"].append(arrys1[vista*1:vista*2])
losses["vistas"].append(vista)

In [ ]:
losses.clear()

In [ ]:
losses

In [ ]:
np.savez('/kaggle/working/entrenamiento_mlp_7.npz', entrenamiento = losses["entrenamiento"], validación = losses["validación"])

### Ploteo de rejilla probabilista

TODO:
- extraer nombre de carpetas
- recorrer carpeta a carpeta
    - extraer el número de capturas realizadas
    - recorrer objetos .npy
    - plotear y almacenar imagen
- Fin


In [ ]:
import os
from classification_nbv import showGrid
import numpy as np

In [ ]:
direccion = 'obj'
x = os.listdir(direccion)
for carpeta in x:
    print(carpeta)
    os.mkdir(direccion + '/' + carpeta + '/ploteos')
    y = os.listdir(direccion + '/' +carpeta + '/RGB/')
    for j in range (0,len(y)):
        if os.path.lexists(direccion + '/' + carpeta + '/Octree/grid_{}.npy'.format(j)) == True:
            occupancy_probs = np.load(direccion + '/' + carpeta + '/Octree/grid_{}.npy'.format(j))
            showGrid(occupancy_probs, direccion + '/' + carpeta + "/ploteos/{}.png".format(j))

### Tabla de latex

In [ ]:
import numpy as np
from tabulate import tabulate

inf = 0 
sup = datos['numero'][0]+1
chamfer = []
distancia = []
for i in range(0,len(datos['numero'])):
    #print(inf,sup)
    if i == 49:
       chamfer.append(datos['chamfer'][sup-1])
       distancia.append(datos['distancias'][sup-1])
    else: 
        chamfer.append(datos['chamfer'][sup-1])
        distancia.append(datos['distancias'][sup-1])
        sup = sup + datos['numero'][i+1] + 1

In [ ]:
obj = datos['obj'].reshape((50,1))
num_vis = datos['numero'].reshape((50,1))
chamfer_np = np.array(chamfer).reshape((50,1))
distancia_np = np.array(distancia).reshape((50,1))
#hs = np.concatenate((obj,num_vis,chamfer_np,distancia_np), axis=1)
hs = np.concatenate((obj,num_vis,chamfer_np,distancia_np), axis=1).tolist()
#hs1 = np.concatenate((num_vis,chamfer_np,distancia_np), axis=1).tolist()
# Define los encabezados de las columnas (opcional)
headers = ["Objeto", "Número de vistas", "Distancia de Chamfer", "Superficie de cobertura"]
#headersm = ["Distancia de Chamfer", "Superficie de cobertura"]

# Genera la tabla LaTeX utilizando tabulate
latex_table = tabulate(hs, headers=headers, tablefmt="latex")
#latex_table1 = tabulate(hs1, headers=headersm, tablefmt="latex")
# Imprime la tabla LaTeX
print(latex_table)
#print(latex_table1)

### Imágenes

In [ ]:
import cv2
import os

In [ ]:
direccion = 'obj'
x = os.listdir(direccion)

for carpeta in x:
    y = os.listdir(os.path.join(direccion, carpeta, 'ploteos'))
    
    # Lista para almacenar las imágenes de cada grupo de 5
    imagenes_grupo = []
    
    for i, imagen in enumerate(y):
        ruta_imagen = os.path.join(direccion, carpeta, 'ploteos', imagen)
        img = cv2.imread(ruta_imagen)
        
        # Verificar si la imagen se cargó correctamente
        if img is not None:
            imagenes_grupo.append(img)
        else:
            print(f"Error al cargar la imagen: {ruta_imagen}")
        
        # Si hemos alcanzado el grupo de 5 imágenes o es la última imagen, concatenamos
        if (i + 1) % 5 == 0 or i == len(y) - 1:
            # Verificar si hay imágenes en el grupo antes de concatenar
            if imagenes_grupo:
                # Concatenar imágenes horizontalmente
                imagen_concatenada = cv2.hconcat(imagenes_grupo)
                
                # Guardar o mostrar la imagen concatenada
                cv2.imwrite(os.path.join(direccion, carpeta, f'concatenadas_{i // 5}.jpg'), imagen_concatenada)
                
                # Limpiar la lista de imágenes del grupo actual
                imagenes_grupo = []
            else:
                print(f"No hay imágenes suficientes para concatenar en el grupo {i // 5} en la carpeta {carpeta}")

In [ ]:
imagen_concatenada

In [ ]:
import open3d as o3d
import numpy as np

def plot_camera(intrinsic_matrix, extrinsic_matrix, width, height, scale=0.1, color=[0.8, 0.2, 0.8]):
    # Crear el marco de la cámara
    camera_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=scale)
    camera_frame.transform(extrinsic_matrix)

    # Calcular las esquinas de la imagen en el espacio de la cámara
    half_width = width / 2
    half_height = height / 2

    # Puntos en coordenadas de píxeles
    pix_points = np.array([
        [0, 0, 1],  # esquina superior izquierda
        [width, 0, 1],  # esquina superior derecha
        [width, height, 1],  # esquina inferior derecha
        [0, height, 1]  # esquina inferior izquierda
    ]).T

    # Convertir a coordenadas de la cámara usando la inversa de la matriz intrínseca
    intrinsic_inv = np.linalg.inv(intrinsic_matrix)
    cam_points = intrinsic_inv @ pix_points

    # Transformar a coordenadas del mundo usando la matriz extrínseca
    world_points = extrinsic_matrix[:3, :3] @ cam_points[:3, :] + extrinsic_matrix[:3, 3:4]

    # Crear la pirámide de visión
    points = [extrinsic_matrix[:3, 3]] + [world_points[:, i] for i in range(4)]
    points = np.array(points)
    lines = [[0, 1], [0, 2], [0, 3], [0, 4], [1, 2], [2, 3], [3, 4], [4, 1]]
    colors = [color for i in range(len(lines))]

    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines)
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)

    return [camera_frame, line_set]

# Ejemplo de uso
# Definir la matriz intrínseca
fov = 30  # campo de visión en grados
width = 100  # ancho de la imagen
height = 100  # alto de la imagen
fov_rad = np.deg2rad(fov)
focal_length = width / (2 * np.tan(fov_rad / 2))

intrinsic_matrix = np.array([
    [focal_length, 0, width / 2],
    [0, focal_length, height / 2],
    [0, 0, 1]
])

# Definir la matriz extrínseca (identidad en este caso, la cámara en el origen)
extrinsic_matrix = np.asarray([
				0.24592144084460449,
				0.0,
				-0.20190031075980167,
				0.0,
				-0.024095638818560146,
				0.096829775473921173,
				-0.029349307061647612,
				0.0,
				0.19310303407831159,
				0.11934423839112578,
				0.23520605883811269,
				0.0,
				0.49999999999999994,
				0.3090169943749474,
				0.80901699437494734,
				1.0
			]).reshape(4,4)

# Crear la geometría de la cámara
camera_geometry = plot_camera(intrinsic_matrix, extrinsic_matrix, width, height)

# Crear una visualización
vis = o3d.visualization.Visualizer()
vis.create_window()

# Agregar geometría de la cámara a la visualización
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.025)
for geom in camera_geometry:
    vis.add_geometry(geom)
    vis.add_geometry(mesh)

# Configuración de la visualización
vis.get_render_option().background_color = np.array([0, 0, 0])  # Fondo negro

# Ejecutar la visualización
vis.run()
vis.destroy_window()